# Exercise 3: Binary search tree

In [ ]:
# Implement binary search tree class
class Tree:
    def __init__(self):
        self._value = None # patient_id; sort by this
        self._data = None # additional patient information
        self.left = None
        self.right = None

    # Insert a value and associated data into the tree
    def add(self, patient_id, data):
        # Check if root is empty
        if self._value is None:
            self._value = patient_id
        else:
            # Check if value is smaller than root
            if patient_id < self._value:
                # Create new node to the left of root
                if self.left is None:
                    self.left = Tree(patient_id)
                # Insert value to the left of root
                else:
                    self.left.add(patient_id)
            # Check if value is larger than root
            elif patient_id > self._value:
                # Create new node to the right of root
                if self.right is None:
                    self.right = Tree(patient_id)
                # Insert value to the right of root
                else:
                    self.right.add(patient_id)

    # Recursive method asking if tree or sub-tree contains a value        
    def __contains__(self, patient_id):
        if self._value == patient_id:
            return True
        elif self.left and patient_id < self._value:
            return patient_id in self.left
        elif self.right and patient_id > self._value:
            return patient_id in self.right
        else:
            return False

In [ ]:
# Test the class
my_tree = Tree()
for patient_id, initials in [(24601, "JV"), (42, "DA"), (7, "JB"), (143, "FR"), (8675309, "JNY")]:
    my_tree.add(patient_id, initials)